<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Financial Fraud Detection with Python and TeradataML
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
    In recent years we have seen a massive increase in Fraud attempts, making fraud detection imperative for Banking and Financial Institutions. Despite countless efforts and human supervision, hundreds of millions of dollars are lost due to fraud. Fraud can happen using various methods, i.e., stolen credit cards, misleading accounting, phishing emails, etc. Due to small cases in significant populations, fraud detection has become more and more challenging. 
    <br>
    <br>
    With ClearScape Analytics, data scientists can use their preferred language, tools and platform to develop models to identify this fraud. Even in large scale operations, users have the guarantee that Vantage can scale to their needs and reduce fraud.</p>
    
<p style = 'font-size:18px;font-family:Arial'><b>Business Values</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Identification of financial fraud in multiple accounts</li>
    <li>Pattern recognition of fraudulent versus normal transactions</li>
    <li>Reduction of money lost due to recovering fraudulent charges</li>
    <li>Improved customer satisfaction and reduction of customer churn</li>
</ul>

<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial'>To maximize the business value of advanced analytic techniques including Machine Learning and Artificial Intelligence, it is estimated that organizations must scale their model development and deployment pipelines to 100s or 1000s of times greater amounts of data, models, or both.
    <br>
    <br>
    ClearScape Analytics provides powerful, flexible end-to-end data connectivity, feature engineering, model training, evaluation, and operational functions that can be deployed at scale as enterprise data assets; treating the products of ML and AI as first-class analytic processes in the enterprise.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the Environment</b>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Standard Libraries
import os
import getpass
import warnings
warnings.filterwarnings("ignore")

# Teradata Libraries
from teradataml import *

# Configuration
spacing_large = " "*95
spacing_small = " "*12
display.max_rows = 5
configure.val_install_location = 'val'

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
from aoa import tmo_create_context
from teradataml import create_context
try:
    tmo_create_context()
except:
    import getpass
    host = input("Host")
    logmech = input("Log Mech")
    username = input("Username")
    password = getpass.getpass()
    create_context(host=host, username=username, password=password, logmech=logmech)
    tmo_create_context()

<p style = 'font-size:16px;font-family:Arial'>We begin running steps with Shift + Enter keys. </p>

In [ ]:
print("Conn successful")

In [ ]:
print("get data start")
# %run -i ../run_procedure.py "call get_data('DEMO_GLM_Fraud_local');" 
execute_sql('''SET query_band='DEMO=16_Fin_Fraud_Det_from_Model_Scheduler.ipynb;' UPDATE FOR SESSION; ''')
execute_sql('''call get_data('DEMO_GLM_Fraud_local');''')
print("get data successful")

In [ ]:
txn_data = DataFrame(in_schema('DEMO_GLM_Fraud', 'transaction_data'))

print(txn_data.shape)
txn_data

<p style = 'font-size:16px;font-family:Arial'>In this simulated scenario, deceptive agents engage in transactions with the objective of taking control of customers' accounts, transferring funds to another account, and ultimately cashing out for profit.</p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.1 How many fraudulent transactions do we have in our dataset?</b></p>

In [ ]:
# There are 92 fraud transactions i.e. 0.14% of fraud transactions in the dataset.
print("No of fraud transactions: %d\nPercentage of fraud transactions: %.2f%%"%(
    txn_data.loc[txn_data.isFraud == 1].shape[0],
    txn_data.loc[txn_data.isFraud == 1].shape[0]/txn_data.shape[0]*100)
)

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.2 How many transactions do we have group by transaction type?</b></p>

In [ ]:
# Filter data for fraud transactions and group by 'type'
transactions_by_type = txn_data.groupby('type').count().get(['type','count_txn_id'])


# Sort by 'count_step' column in descending order
transactions_by_type = transactions_by_type.sort('count_txn_id', ascending = False)

transactions_by_type = transactions_by_type.assign(
    type_int = case([
        (transactions_by_type.type == 'CASH_IN', 0),
        (transactions_by_type.type == 'CASH_OUT', 1),
        (transactions_by_type.type == 'DEBIT', 2),
        (transactions_by_type.type == 'PAYMENT ', 3),
        (transactions_by_type.type == 'TRANSFER', 4),
    ])
)

In [ ]:
transactions_by_type.plot(
    x = transactions_by_type.type_int,
    y = transactions_by_type.count_txn_id,
    kind = 'bar',
    legend = ['Count by Type'],
    ylabel = 'Count of Transactions',
    xlabel = spacing_small.join(sorted(list(transactions_by_type[['type']].get_values().flatten()))),
    title = "Number of Transactions per Transaction Type"
)

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.3 How many fraudulent transactions do we have group by transaction type?</b></p>

In [ ]:
# Filter data for fraud transactions and group by 'type'
fraud_transactions_by_type = txn_data.loc[txn_data.isFraud == 1].groupby('type').count().get(['type','count_txn_id'])

# Sort by 'count_step' column in descending order
fraud_transactions_by_type = fraud_transactions_by_type.sort('count_txn_id', ascending = False)

fraud_transactions_by_type = fraud_transactions_by_type.assign(
    total_fraud = txn_data.loc[txn_data.isFraud == 1].shape[0],
    type_int = case([(fraud_transactions_by_type.type == 'TRANSFER', 0)], else_ = 1)
)

In [ ]:
fraud_transactions_by_type.plot(
    x = fraud_transactions_by_type.type_int,
    y = [fraud_transactions_by_type.total_fraud, fraud_transactions_by_type.count_txn_id],
    kind = 'bar',
    figsize = (800, 500),
    legend = ['Total Fraud', 'Count by Type'],
    ylabel = 'Count of Fraud Transactions',
    xlabel = 'TRANSFER' + spacing_large + 'CASH_OUT',
    title = "Number of Fraud Transactions by Transaction Type"
)

<p style = 'font-size:16px;font-family:Arial'>From the above result, we can see that out of the 92 fraud transactions, 47 are from transaction type "TRANSFER" and 45 are from "CASH_OUT".</p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.4 What percentage of fraudulent transactions do we have where transaction amount is equal to old balance in the origin account?</b></p>

<p style = 'font-size:16px;font-family:Arial'>This might be the case where the fraudster emptied the account of the victim.</p>

In [ ]:
print("No of cleanout fraud transactions: %d\nPercentage of cleanout fraud transactions: %.2f%%"%(
    txn_data.loc[txn_data['amount'] == txn_data.oldbalanceOrig].loc[txn_data['isFraud'] == 1].shape[0],
    txn_data.loc[txn_data['amount'] == txn_data.oldbalanceOrig].loc[txn_data['isFraud'] == 1].shape[0] / txn_data.loc[txn_data.isFraud == 1].shape[0]*100)
)

<p style = 'font-size:16px;font-family:Arial'>From the above result, we can see that out of 92 Fraud transactions, the amount involved in 90 fraud transactions was equal to the total balance in the account. </p>

<hr style="height:1px;border:none;">
<p style = 'font-size:16px;font-family:Arial'><b>Below are some insights about the dataset:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>We have 92 fraud transactions, which account for 0.14% of the dataset.</li>
    <li>Out of these 92 fraud transactions, 47 are of type TRANSFER, and 45 are of type CASH_OUT.</li>
    <li>Approximately 97.83% of our fraud transactions have a transaction amount equal to oldbalanceOrig, indicating account cleanout.</li>
    <li>About 71.74% of our fraud transactions have the recipient's old balance as zero.</li>
    <li>The isFlaggedFraud indicator is correct only two times among our 92 fraud transactions.</li>
</ol>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.5 Univariate statistics</b></p>

<p style = 'font-size:16px;font-family:Arial'>The describe funtion computes the count, mean, std, min, percentiles, and max for numeric columns.</p>

In [ ]:
txn_data.describe()

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.6 Checking for Null Values</b></p>
<p style = 'font-size:16px;font-family:Arial'>The ColumnSummary() function can be used to take a quick look at the columns, their datatypes, and summary of NULLs/non-NULLs for a given table.</p>

In [ ]:
colsum = ColumnSummary(
    data  = txn_data,
    target_columns = [':']
)
colsum.result

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.7 Checking for Outliers</b></p>
<p style = 'font-size:16px;font-family:Arial'>The OutlierFilterFit() function calculates the lower percentile, upper percentile, count of rows and median for all the "target_columns" provided by the user. These metrics for each column help the function OutlierTransform() detect outliers in data.</p>

<p style = 'font-size:16px;font-family:Arial'>Here we are using teradataml syntax for the function. The same can be achived using the following SQL as well.</p>

<code>SELECT * FROM TD_OutlierFilterFit(
    ON "DEMO_GLM_Fraud"."transaction_data" AS InputTable
    OUT TABLE OutputTable("DEMO_USER"."Outlier_output")
    USING
    TargetColumns('amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig')
) as dt;</code>

<p style = 'font-size:14px;font-family:Arial'><b><i>*Please note that both the versions run in-database and there is no data transfer involved.</i></b></p>

In [ ]:
fit_object = OutlierFilterFit(
    data = txn_data,
    target_columns = ['amount','newbalanceOrig', 'oldbalanceDest','newbalanceDest','oldbalanceOrig']
)

res = fit_object.transform(data = txn_data).result

In [ ]:
print(f"Rows before removing outliers: {txn_data.shape[0]}\n\
Rows after removing outliers: {res.shape[0]}\n\
Total outliers: {txn_data.shape[0] - res.shape[0]}")

In [ ]:
outliers = td_minus([txn_data, res])
outliers

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Data Preparation</b>

<p style='font-size:16px;font-family:Arial'><b>We'll perform the following steps:</b></p>
<ul style='font-size:16px;font-family:Arial'>
    <li>We will one-hot encode the categorical "type" column.</li>
    <li>We will perform feature scaling using ScaleFit and ScaleTransform on numerical columns.</li>
    <li>We will split the data into training and testing datasets (80:20 split).</li>
</ul>

<p style='font-size:16px;font-family:Arial'>We perform feature scaling during data pre-processing to handle highly varying magnitudes, values, or units. If feature scaling is not done, then a machine learning algorithm tends to weigh greater values higher and consider smaller values as lower ones, regardless of the unit of the values.</p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>4.1 Drop redundant columns</b></p>
<p style = 'font-size:16px;font-family:Arial'>We don't need nameDest, nameOrigin, and isFlaggedFraud for model training as they do not impact the outcome. We have txn_id to uniquely identify each transaction.</p>

In [ ]:
txn_data = txn_data.drop(['nameDest', 'nameOrig', 'isFlaggedFraud'], axis = 1)
txn_data

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>4.2 One-hot encoding</b></p>
<p style='font-size:16px;font-family:Arial'>
Here, we are one-hot encoding the "type" column. We find one-hot encoding necessary in many cases to represent categorical variables as binary values, enable numerical processing, ensure feature independence, handle non-numeric data, and improve the performance and interpretability of our machine learning models.
</p>

In [ ]:
txn_type_encoder = OneHotEncoder(
    values = ["CASH_IN", "CASH_OUT", "DEBIT", "PAYMENT", "TRANSFER"],
    columns = "type"
)

retain = Retain(
    columns = ['step', 'amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig', 'isFraud']
)

obj = valib.Transform(
    data = txn_data,
    one_hot_encode = txn_type_encoder,
    retain = retain,
    index_columns = 'txn_id'
)
txn_trans = obj.result
txn_trans

<p style = 'font-size:16px;font-family:Arial'>The above output shows that we have transformed the data into a transfromed dataset.</p>

In [ ]:
copy_to_sql(txn_trans, table_name = 'clean_data', if_exists = 'replace')

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Create training and testing datasets in Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We'll create two datasets for training and testing in the ratio of 80:20.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(
    data = txn_trans,
    id_column = "txn_id",
    train_size = 0.80,
    test_size = 0.20,
    seed = 25
)

df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

print("Training Set = " + str(df_train.shape[0]) + ". Testing Set = " + str(df_test.shape[0]))

In [ ]:
copy_to_sql(df_train, table_name = 'clean_data_train', if_exists = 'replace')
copy_to_sql(df_test, table_name = 'clean_data_test', if_exists = 'replace')

In [ ]:
df_train

<p style = 'font-size:16px;font-family:Arial'>The above output shows that we have transformed the data into a scaled dataset. Scaling our data makes it easy for our model to learn and understand the problem.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>6. In-Database XGBoost model training</b>

<p style = 'font-size:16px;font-family:Arial'>The XGBoost() function, also known as eXtreme Gradient Boosting, is an implementation of the gradient boosted decision tree algorithm designed for speed and performance. It has recently been dominating applied machine learning.</p>
<p style = 'font-size:16px;font-family:Arial'>In gradient boosting, each iteration fits a model to the residuals (errors) of the previous iteration to correct the errors made by existing models. The predicted residual is multiplied by this learning rate and then added to the previous prediction. Models are added sequentially until no further improvements can be made. It is called gradient boosting because it uses a gradient descent algorithm to minimize the loss when adding new models.</p>

<p style = 'font-size:16px;font-family:Arial'>Here we are using teradataml syntax for the function. The same can be achived using the following SQL as well.</p>

<code>SELECT * FROM TD_XGBoost(
	ON "DEMO_USER"."clean_data_train" AS "input"
	PARTITION BY ANY
	USING InputColumns('amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig','CASH_IN_type','CASH_OUT_type','DEBIT_type','PAYMENT_type','TRANSFER_type')
	ResponseColumn('isFraud')
	MaxDepth(7)
	Seed(42)
	ModelType('Classification')
	RegularizationLambda(120.0)
	ShrinkageFactor(0.1)
) as sqlmr</code>

<p style = 'font-size:14px;font-family:Arial'><b><i>*Please note that both the versions run in-database and there is no data transfer involved.</i></b></p>

In [ ]:
cols = df_train.columns
cols.remove('txn_id')
cols.remove('step')
cols.remove('isFraud')

In [ ]:
XGBoost_out = XGBoost(
    data=df_train,
    input_columns=cols,
    response_column = 'isFraud',
    lambda1 = 120.0,
    model_type='Classification',
    seed=42,
    shrinkage_factor=0.1,
    max_depth=7
)

In [ ]:
XGBoost_out.output_data

<p style = 'font-size:16px;font-family:Arial'>The function output is a trained XGBoost model, and we can input it to the XGBoostPredict() function for prediction.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>7. In-Database XGBoost model scoring</b>

<p style = 'font-size:16px;font-family:Arial'>The XGBoostPredict() function runs the predictive algorithm based on the model generated by XGBoost(). The XGBoost() function, also known as eXtreme Gradient Boosting, performs classification or regression analysis on datasets.</p>
<p style = 'font-size:16px;font-family:Arial'>
When using the function, we should provide only numeric features. We need to convert the categorical features to numeric values before prediction.</p>

In [ ]:
XGBoostPredict_out = XGBoostPredict(
    newdata=df_test,
    object=XGBoost_out.result,
    model_type='Classification',
    id_column='txn_id',
    object_order_column=['task_index', 'tree_num',
                       'iter', 'tree_order'],
    accumulate='isFraud',
    output_prob=True,
    output_responses=['0', '1']
).result

In [ ]:
XGBoostPredict_out

<p style = 'font-size:16px;font-family:Arial'>The output above shows our prob_1, i.e., the transaction is fraud, and prob_0, i.e., the transaction is not a fraud. We use these probabilities in our prediction column to assign a class label.</p>

In [ ]:
combined_df = df_test.join(XGBoostPredict_out, on='txn_id', lsuffix='test', rsuffix='pred')

In [ ]:
combined_df[combined_df['Prediction']==1]

In [ ]:
out = XGBoostPredict_out.assign(Prediction = XGBoostPredict_out.Prediction.cast(type_ = BYTEINT))
out = out.assign(Prediction = out.Prediction.cast(type_ = VARCHAR(2)))
out = out.assign(isFraud = out.isFraud.cast(type_ = VARCHAR(2)))

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(
    data = out,
    observation_column = 'isFraud',
    prediction_column = 'Prediction',
    labels = ['0', '1']
)

In [ ]:
ClassificationEvaluator_obj.output_data.head(10)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>8. Visualize the results (ROC curve and AUC)</b>

<p style = 'font-size:16px;font-family:Arial'>We create the ROC curve, which is a graph between TPR (True Positive Rate) and FPR (False Positive Rate). We use the area under the ROC curve as a metric to evaluate how well our model can distinguish between positive and negative classes. A higher AUC indicates better performance in distinguishing between the positive and negative categories. We generally consider an AUC above 0.75 as decent.</p>

In [ ]:
from teradataml import ROC

roc_out = ROC(
    probability_column = '"Prob_1"',
    observation_column = "isFraud",
    positive_class = "1",
    data = XGBoostPredict_out,
    num_thresholds=300
)

roc_out

In [ ]:
# Assigning new index column
roc_out.result = roc_out.result.assign(row = 1)
# Changing the index label.
roc_out.result._index_label = ["row"]
auc = roc_out.result.get_values()[0][0]

figure = Figure(width=500, height=400, heading="Receiver Operating Characteristic (ROC) Curve")

plot = roc_out.output_data.plot(
    x=roc_out.output_data.fpr,
    y=[roc_out.output_data.tpr, roc_out.output_data.fpr],
    xlabel='False Positive Rate',
    ylabel='True Positive Rate',
    color='carolina blue',
    figure=figure,
    legend=[f'XGBoost AUC = {round(auc, 4)}', 'AUC Baseline'],
    legend_style='lower right',
    grid_linestyle='--',
    grid_linewidth=0.5,
    linestyle = ['-', '--']
)

plot.show()

<p style = 'font-size:16px;font-family:Arial'>Looking at the above ROC Curve, we can confidently say that our model has performed well on testing data. The AUC value is above 0.75 and resonates with our understanding that the model is performing well.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Conclusion</b></p>

<p style = 'font-size:16px;font-family:Arial'>In this demonstration, we have illustrated a simplified - but complete - overview of how we can implement a typical machine learning workflow completely inside the database using Vantage. This allows us to leverage Vantage's operational scale, power, and stability.</p>

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>9. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
tables = ['clean_data', 'clean_data_train', 'clean_data_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
print("remove data start")
# %run -i ../run_procedure.py "call get_data('DEMO_GLM_Fraud_local');" 
execute_sql('''call remove_data('DEMO_GLM_Fraud');''')
print("remove data successful")

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>